In [11]:
from dotenv import load_dotenv
import os
from pydantic import BaseModel, Field
from langchain_openai   import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain.agents import create_tool_calling_agent, AgentExecutor
from tools import search_tool
from langchain.chains import LLMMathChain
from langchain.tools import Tool

In [12]:
from langchain_openai import ChatOpenAI

load_dotenv()

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    max_retries=2,
    # api_key="...",
    # base_url="...",
    # organization="...",
    # other params...
)

class PydanticModelResponse(BaseModel):
    topic: str
    summary: str
    sources : list[str]
    tools_used: list[str]



parser = PydanticOutputParser(pydantic_object=PydanticModelResponse)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that provide the flight information \
                        and travel tips based on the user's query. Where ever you have to to calculation you will use \
                        Calculator tool. You will use the Search tool to find information on the web."),
        ("human", "{input}"),
        ("placeholder", "{chat_history}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
).partial(format_instructions=parser.get_format_instructions())

problem_chain = LLMMathChain.from_llm(
    llm=llm)
math_tool = Tool.from_function(name="Calculator",
                func=problem_chain.run,
                 description="Useful for when you need to answer questions about math. \
                    This tool is only for math questions and nothing else. Only input math expressions.")

tools  = [search_tool, math_tool]

agent = create_tool_calling_agent(
    llm=llm,
    prompt=prompt,
    tools = tools
)

agent_executor = AgentExecutor(agent = agent, tools = tools, verbose=True)



In [14]:
response = agent_executor.invoke({
    "input": "What is the Price of Flight to France from New Delhi, India on 23rd Dec'2025? calculate the flight fare in dubai currency.",
})



> Entering new AgentExecutor chain...

Invoking: `search` with `Flight price from New Delhi, India to France on 23rd Dec 2025`


Book New Delhi to Paris flight tickets at the cheapest price starting at ₹21905. Compare approximately 55 flights of major airlines that fly between New Delhi and Paris. The shortest flight duration is 9h 35m. ixigo also provides information like flight fares, flight timings, flight schedule changes & stops (if any), etc. How to find cheap flights to Paris (PARI) from New Delhi (IDEL) in 2025. Looking for cheap tickets from New Delhi to Paris? Return tickets start from ₹ 37,040 and one-way flights to Paris from New Delhi start from ₹ 19,768. Here are a few tips on how to secure the best flight price and make your journey as smooth as possible. Simply hit ... 16 Mar 2025. Air France lowest fare. ... First, and Business Class on DEL to POX Flights. The date of the cheapest flight price for a New Delhi Paris flight ticket is undefined. ... arrives in Paris at 

In [6]:
response["output"]

'The price of a flight from New Delhi, India to France on 23rd Dec 2025 varies depending on the airline and class of travel. The cheapest price for a flight ticket on this route starts at ₹21,905. Air France and Air India are among the airlines that operate flights between New Delhi and France.'